In [1]:
from sqlalchemy import create_engine
import pandas as pd
from modules.backtest import cumplot, mvplot, regress
import datapane as dp

In [2]:
with create_engine("sqlite:///files/mydata.db").connect() as conn:
    rets = pd.read_sql("select * from model1", conn)
    rets["market"] = pd.read_sql("select market from market", conn)
    rets["rf"] = pd.read_sql("select rf from market", conn)

In [3]:
text = """
    The features are roeq and mom12m.  The feature pipeline is (QuantileTransformer, 
    PolynomialFeatures, QuantileTransformer).  The target is the return minus the
    market return and ias transformed with QuantileTransformer.  Random forests with
    max_depth=4 are trained at five-year intervals and used to predict monthly.  The best
    and worst returns are the equally weighted returns of the top 100 and bottom 100 stocks
    each month, respectively.
"""

figa = cumplot(rets)
figb = mvplot(rets)
tbla = regress(rets, "best")
tblb = regress(rets, "worst")

In [11]:
pg = dp.Page(
    title="Random Forest",
    blocks = [
        "## Random Forest Model",
        "Kerry Back, November 28, 2022",
        text,
        dp.Select(
            dp.Plot(
                figa, 
                caption="Cumulative Returns",
                label="Cumulative returns"
            ),
            dp.DataTable(
                tbla, 
                caption="""
                    Regression of Best-RF on Mkt-RF.  The alpha estimate is
                    in percent per year.
                """,
                label="Alpha and beta of best portfolio"
            ),
            dp.DataTable(
                tblb, 
                caption="""
                    Regression of Worst-RF on Mkt-RF.  The alpha estimate is
                    in percent per year.
                """,
                label="Alpha and beta of worst portfolio"
            ),
            dp.Plot(
                figb, 
                caption="Mean-Variance Analysis",
                label="Mean-variance analysis"
            ),
        ),
    ]
)

app = dp.App(
    pg,
)
app.save("files/dashboard.html")


c:\Users\kerry\repos\2022-638-binder\venv\lib\site-packages\datapane\common\df_processor.py:28: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



App saved to ./files/dashboard.html

If you want to create an html file with separate pages for different models, you can create each page as in the above and then use something like

    app = dp.App(
        pg1, 
        pg2,
        pg3
    )



datapane.com will host your html file for free.  Just create an account, get your login token and add the following two lines to your notebook.  You will then get a link that you can share.

    !datapane login --token=whateverYourTokenIs
    app.upload(name="WhateverYouWantToCallIt")

In [5]:
# Example
# 
# !datapane login --token=
# app.upload(name="Investment Strategy Backtests")